In [1]:
import os 
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from typing import Optional 
from langchain_core.pydantic_v1 import BaseModel,Field 

class Person(BaseModel):
    """ 
    Information about a person.
    """

    name: Optional[str] = Field(
        default=None,description="The name of the person"
    )

    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )

    country: Optional[str] = Field(
        default=None,description="The country of the person if known"
    )

### Define the Extractor

In [4]:
from typing import Optional 
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel,Field 

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        "You are an expert extraction algorithm"
        "Only extract relevant information from the text"
        "If you do not know the value of an attribute asked to extract"
        "return null for the attributes value." 
        ),
        ("human","{text}")
    ]
)

In [5]:
chain = prompt | llm.with_structured_output(schema=Person)

In [6]:
comment = "Hello, Very Good Morning. Mr. Gorantla Sai Krishna is a great data scientist from the India"

chain.invoke({"text":comment})

Person(name='Gorantla Sai Krishna', lastname=None, country='India')

### Extraction of a list of entities instead of a single entity

In [7]:
from typing import Optional, List
from langchain_core.pydantic_v1 import BaseModel,Field 

class Person(BaseModel):
    """ 
    Information about a person.
    """

    name: Optional[str] = Field(
        default=None,description="The name of the person"
    )

    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )

    country: Optional[str] = Field(
        default=None,description="The country of the person if known"
    )

class Data(BaseModel):
    """Extracted data about people."""

    people: List[Person]

In [8]:
chain_2 = prompt | llm.with_structured_output(schema=Data)

In [9]:
comment = "Hello, Very Good Morning. Mr. Gorantla Sai Krishna  and sai nishith varaganti are a great data scientists from the India"

chain_2.invoke({"text":comment})

Data(people=[Person(name='Gorantla Sai Krishna', lastname=None, country='India'), Person(name='sai nishith varaganti', lastname=None, country='India')])

### Sentiment Analysis Tool

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field 

tagging_prompt = ChatPromptTemplate.from_template(
    """ 
    Extract the desired information  from the following passage.

    Only extract the properties mentioned in the 'Classification' function.

    passage:
    {input}
    """
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text",enum=["positive","Negative","Neutral"])
    political_tendency: str = Field(description="The political tendency of the user")
    language: str = Field(description="The language the text is written in ")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0).with_structured_output(Classification)

tagging_chain = tagging_prompt | llm 

modi = "ndia believes in taking everyone together. Nepal was rattled with an earthquake. We provided assistance to them. We did so, on the grounds of humanity. Be it Yemen or Maldives, we tried to provide assistance at our best. Humanity is our central inspiration. -PM Modi at inauguration of Pravasi Bharatiya Kendra"

tagging_chain.invoke({"input":modi})

Classification(sentiment='positive', political_tendency='right-wing', language='English')